In [140]:
def DichotomyOneCalc(func, a, b, delta = 0.1, eps = 0.001):
    while b - a > delta:
        x1 = - eps + (b + a) / 2. 
        x2 = eps + (b + a) / 2. 
        if func(x1) < func(x2):
            b = x2
        else:
            a = x1
    return [a, b]

In [141]:
def test_func(x):
    return np.abs(np.log(x))
result_interval = DichotomyOneCalc(test_func, 0.1, 30000, 0.001,0.0000001)

In [142]:
result_interval

[0.9994310915410164, 1.000325358232409]